In [ ]:
Gostaria de aplicar um algoritmo de agrupamento na seguinte base de dados:
https://www02.smt.ufrj.br/~offshore/mfs/page_01.html
Nela foram feitas algumas medições de vibrações do rotor do motor a 50 Hz. A estrutura de pasta que foi montada segue o seguinte arquivo anexado. Gostaria de primeiramente carregar esses dados usando polars dataframes

-----------------

Para carregar esses dados usando Polars DataFrames, você pode criar uma função que percorra as pastas e leia os arquivos CSV correspondentes. Aqui está um exemplo de código em Python:


In [ ]:
import polars as pl
import os

# Função para carregar os dados
def load_vibration_data(base_path):
    data = []
    labels = []
    
    # Percorrer as pastas
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                # Ler o arquivo CSV
                df = pl.read_csv(file_path)
                # Adicionar os dados ao dataframe principal
                data.append(df)
                # Extrair o label com base no nome do diretório
                label = os.path.basename(os.path.dirname(file_path))
                labels.append(label)
    
    # Combinar os dados em um único dataframe
    combined_df = pl.concat(data)
    return combined_df, labels

# Caminho base para os dados
base_path = "Data"  # Substitua pelo caminho correto dos seus dados

# Carregar os dados
combined_df, labels = load_vibration_data(base_path)

# Exibir as primeiras linhas
print(combined_df.head())


Os dados consistem em séries temporais de vibrações dos motores. Há alguma forma de gerar atributos que consigam caracterizar bem os dados para aplicar um algoritmo de agrupamento capaz de detectar quando há uma anomalia?

------------------------

Sim, para caracterizar séries temporais de vibrações de motores e aplicar algoritmos de agrupamento para detectar anomalias, você pode gerar atributos derivados das séries temporais que capturem padrões relevantes para análise. Abaixo estão algumas categorias de atributos úteis, além de métodos para extraí-los:

1. Estatísticas Básicas
Média, mediana, variância e desvio padrão da amplitude.
Amplitude máxima e mínima.
Curtose e assimetria (skewness).
Energia da série temporal.
Exemplo em Polars:

In [ ]:
def basic_statistics(df):
    stats = df.select([
        pl.col("signal").mean().alias("mean"),
        pl.col("signal").std().alias("std_dev"),
        pl.col("signal").min().alias("min"),
        pl.col("signal").max().alias("max"),
        pl.col("signal").kurtosis().alias("kurtosis"),
        pl.col("signal").skew().alias("skewness")
    ])
    return stats


2. Atributos no Domínio da Frequência
Transformar os dados para o domínio da frequência pode revelar padrões ocultos associados a vibrações específicas:

Frequências dominantes: Componentes de maior magnitude no espectro.
Energia em bandas específicas: Soma da potência em faixas de frequência (e.g., baixas, médias, altas).
Transformada de Fourier (FFT): Frequência fundamental e harmônicas.
Exemplo com scipy:

In [ ]:
import numpy as np
from scipy.fftpack import fft

def frequency_features(signal, sampling_rate):
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1/sampling_rate)
    fft_values = np.abs(fft(signal))
    
    # Seleção de características
    dominant_freq = freq[np.argmax(fft_values)]
    energy = np.sum(fft_values**2)  # Energia total
    return {
        "dominant_frequency": dominant_freq,
        "frequency_energy": energy
    }


3. Características Derivadas do Tempo
Capturam comportamentos ao longo do tempo:

Autocorrelação: Mede a relação do sinal consigo mesmo em diferentes atrasos (lags).
Entropia de Permutação: Avalia a complexidade do sinal.
Taxa de cruzamento de zero: Frequência com que o sinal cruza o zero (relacionado à oscilação).
Exemplo de cruzamento de zero:

In [ ]:
def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)


4. Características no Domínio do Tempo-Frequência
Transformada Wavelet: Identifica padrões transitórios.
Espectrograma: Representação de frequência ao longo do tempo. Essas técnicas são úteis para sinais com características que mudam no tempo.

5. Técnicas Avançadas
Redução de Dimensionalidade: Usar PCA (Principal Component Analysis) para reduzir a dimensionalidade das características derivadas.
Aprendizado de Representação:
Redes Neurais (e.g., autoencoders).
Extração de embeddings usando técnicas de aprendizado profundo.

Pipeline para Detectar Anomalias
Pré-processamento: Normalizar os dados e lidar com ruído.
Geração de Atributos: Usar as categorias acima para transformar os dados.
Agrupamento: Aplicar algoritmos como K-Means, DBSCAN ou Isolation Forest nos atributos derivados.
Validação: Avaliar os resultados manualmente ou com labels de anomalias, caso existam.
Se quiser, posso implementar parte desse pipeline em Python para os seus dados, como a extração de atributos estatísticos ou baseados em frequência.

Sim por favor, faça a implementação completa

--------------------------

Aqui está a implementação completa de um pipeline para caracterizar séries temporais de vibrações e aplicar algoritmos de agrupamento para detectar anomalias:

In [ ]:
import os
import polars as pl
import numpy as np
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA


# Step 1: Load Data
def load_vibration_data(base_path):
    data = []
    labels = []

    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                df = pl.read_csv(file_path)
                data.append(df)
                label = os.path.basename(os.path.dirname(file_path))
                labels.append(label)

    return data, labels


# Step 2: Extract Features
def extract_features(df, sampling_rate):
    signal = df["signal"].to_numpy()

    # Basic Statistics
    mean = np.mean(signal)
    std_dev = np.std(signal)
    min_val = np.min(signal)
    max_val = np.max(signal)
    kurtosis = pl.Series(signal).kurtosis()
    skewness = pl.Series(signal).skew()

    # Frequency Domain Features
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1 / sampling_rate)
    fft_values = np.abs(fft(signal))
    dominant_freq = freq[np.argmax(fft_values)]
    energy = np.sum(fft_values**2)

    # Time Domain Features
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zero_crossing_rate = len(zero_crossings) / len(signal)

    return {
        "mean": mean,
        "std_dev": std_dev,
        "min": min_val,
        "max": max_val,
        "kurtosis": kurtosis,
        "skewness": skewness,
        "dominant_frequency": dominant_freq,
        "frequency_energy": energy,
        "zero_crossing_rate": zero_crossing_rate,
    }


# Step 3: Build Feature Matrix
def build_feature_matrix(data, sampling_rate):
    features = []
    for df in data:
        features.append(extract_features(df, sampling_rate))

    return pl.DataFrame(features)


# Step 4: Preprocessing and Clustering
def cluster_data(features):
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Dimensionality Reduction (optional)
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(scaled_features)

    # DBSCAN Clustering
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    clusters = dbscan.fit_predict(reduced_features)

    return reduced_features, clusters


# Step 5: Visualize Clusters
def visualize_clusters(reduced_features, clusters):
    import matplotlib.pyplot as plt

    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap="viridis")
    plt.colorbar(label="Cluster Label")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("DBSCAN Clustering Results")
    plt.show()


# Main Pipeline
if __name__ == "__main__":
    # Base path to data
    base_path = "motorvibration/Data"  # Update with the correct path

    # Sampling rate (assumed, update as needed)
    sampling_rate = 50

    # Load Data
    data, labels = load_vibration_data(base_path)
    print(data)

    # Build Feature Matrix
    features = build_feature_matrix(data, sampling_rate)

    # Clustering
    reduced_features, clusters = cluster_data(features)

    # Visualize Results
    visualize_clusters(reduced_features, clusters)

    # Display cluster assignment
    print("Cluster Assignments:", clusters)


Cada arquivo csv apresenta 8 colunas, elas apontam as medições na imagem. Portanto as colunas serão:
dataset_columns_name = ["tachometer", "underhang_1", "underhang_2", "underhang_3", "overhang_1", "overhang_2", "overhang_3", "microphone"]
Poderia também colocar uma barra de carregamento para indicar que a base está sendo carregada? 

--------------

Adicionei uma barra de carregamento usando a biblioteca tqdm para indicar o progresso do carregamento dos arquivos e da extração de características. Além disso, atualizei as colunas do dataset para refletir os nomes fornecidos. Certifique-se de que a biblioteca tqdm esteja instalada (pip install tqdm) antes de executar o código. Se precisar de ajustes adicionais, é só avisar!

In [ ]:
import os
import polars as pl
import numpy as np
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from tqdm import tqdm

# Step 1: Load Data
def load_vibration_data(base_path):
    dataset_columns_name = ["tachometer", "underhang_1", "underhang_2", "underhang_3", "overhang_1", "overhang_2", "overhang_3", "microphone"]
    data = []
    labels = []

    for root, dirs, files in os.walk(base_path):
        for file in tqdm(files, desc="Loading files"):
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                df = pl.read_csv(file_path, has_header=True, new_columns=dataset_columns_name)
                data.append(df)
                label = os.path.basename(os.path.dirname(file_path))
                labels.append(label)

    return data, labels

# Step 2: Extract Features
def extract_features(df, sampling_rate):
    signal = df["microphone"].to_numpy()

    # Basic Statistics
    mean = np.mean(signal)
    std_dev = np.std(signal)
    min_val = np.min(signal)
    max_val = np.max(signal)
    kurtosis = pl.Series(signal).kurtosis()
    skewness = pl.Series(signal).skew()

    # Frequency Domain Features
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1 / sampling_rate)
    fft_values = np.abs(fft(signal))
    dominant_freq = freq[np.argmax(fft_values)]
    energy = np.sum(fft_values ** 2)

    # Time Domain Features
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zero_crossing_rate = len(zero_crossings) / len(signal)

    return {
        "mean": mean,
        "std_dev": std_dev,
        "min": min_val,
        "max": max_val,
        "kurtosis": kurtosis,
        "skewness": skewness,
        "dominant_frequency": dominant_freq,
        "frequency_energy": energy,
        "zero_crossing_rate": zero_crossing_rate,
    }

# Step 3: Build Feature Matrix
def build_feature_matrix(data, sampling_rate):
    features = []
    for df in tqdm(data, desc="Extracting features"):
        features.append(extract_features(df, sampling_rate))

    return pl.DataFrame(features)

# Step 4: Preprocessing and Clustering
def cluster_data(features):
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Dimensionality Reduction (optional)
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(scaled_features)

    # DBSCAN Clustering
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    clusters = dbscan.fit_predict(reduced_features)

    return reduced_features, clusters

# Step 5: Visualize Clusters
def visualize_clusters(reduced_features, clusters):
    import matplotlib.pyplot as plt

    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap="viridis")
    plt.colorbar(label="Cluster Label")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("DBSCAN Clustering Results")
    plt.show()

# Main Pipeline
if __name__ == "__main__":
    # Base path to data
    base_path = "Data"  # Update with the correct path

    # Sampling rate (update as needed)
    sampling_rate = 50000

    # Load Data
    data, labels = load_vibration_data(base_path)

    # Build Feature Matrix
    features = build_feature_matrix(data, sampling_rate)

    # Clustering
    reduced_features, clusters = cluster_data(features)

    # Visualize Results
    visualize_clusters(reduced_features, clusters)

    # Display cluster assignment
    print("Cluster Assignments:", clusters)


Implementei a seguinte função para facilitar a listagem dos diversos arquivos que existem:

import os
import polars as pl
import numpy as np
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from tqdm import tqdm
from glob import glob

# Step 1: List Files
def list_files(dataset_path):
    csv_files_list = []

    folders_list = os.listdir(dataset_path)
    for folder in folders_list:
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            sub_folders_list = os.listdir(folder_path)
            for sub_folder in sub_folders_list:
                sub_folder_path = os.path.join(folder_path, sub_folder)
                if os.path.isdir(sub_folder_path):
                    list_csv = glob(f"{sub_folder_path}/*.csv")
                    for csv_file in list_csv:
                        csv_files_list.append((csv_file, csv_file))
                elif sub_folder_path.endswith(".csv"):
                    csv_files_list.append((sub_folder_path, sub_folder_path))

    return csv_files_list

# Step 2: Load Data
def load_vibration_data(base_path):
    dataset_columns_name = ["tachometer", "underhang_1", "underhang_2", "underhang_3", "overhang_1", "overhang_2", "overhang_3", "microphone"]
    data = []
    labels = []

    for root, dirs, files in os.walk(base_path):
        for file in tqdm(files, desc="Loading files"):
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                df = pl.read_csv(file_path, has_header=True, new_columns=dataset_columns_name)
                data.append(df)
                label = os.path.basename(os.path.dirname(file_path))
                labels.append(label)

    return data, labels

# Step 3: Extract Features
def extract_features(df, sampling_rate):
    signal = df["microphone"].to_numpy()

    # Basic Statistics
    mean = np.mean(signal)
    std_dev = np.std(signal)
    min_val = np.min(signal)
    max_val = np.max(signal)
    kurtosis = pl.Series(signal).kurtosis()
    skewness = pl.Series(signal).skew()

    # Frequency Domain Features
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1 / sampling_rate)
    fft_values = np.abs(fft(signal))
    dominant_freq = freq[np.argmax(fft_values)]
    energy = np.sum(fft_values ** 2)

    # Time Domain Features
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zero_crossing_rate = len(zero_crossings) / len(signal)

    return {
        "mean": mean,
        "std_dev": std_dev,
        "min": min_val,
        "max": max_val,
        "kurtosis": kurtosis,
        "skewness": skewness,
        "dominant_frequency": dominant_freq,
        "frequency_energy": energy,
        "zero_crossing_rate": zero_crossing_rate,
    }

# Step 4: Build Feature Matrix
def build_feature_matrix(data, sampling_rate):
    features = []
    for df in tqdm(data, desc="Extracting features"):
        features.append(extract_features(df, sampling_rate))

    return pl.DataFrame(features)

# Step 5: Preprocessing and Clustering
def cluster_data(features):
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Dimensionality Reduction (optional)
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(scaled_features)

    # DBSCAN Clustering
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    clusters = dbscan.fit_predict(reduced_features)

    return reduced_features, clusters

# Step 6: Visualize Clusters
def visualize_clusters(reduced_features, clusters):
    import matplotlib.pyplot as plt

    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap="viridis")
    plt.colorbar(label="Cluster Label")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("DBSCAN Clustering Results")
    plt.show()

# Main Pipeline
if __name__ == "__main__":
    # Base path to data
    base_path = "Data"  # Update with the correct path

    # Sampling rate (update as needed)
    sampling_rate = 50000

    # List Files
    file_list = list_files(base_path)
    print("Files found:", file_list)

    # Load Data
    data, labels = load_vibration_data(base_path)

    # Build Feature Matrix
    features = build_feature_matrix(data, sampling_rate)

    # Clustering
    reduced_features, clusters = cluster_data(features)

    # Visualize Results
    visualize_clusters(reduced_features, clusters)

    # Display cluster assignment
    print("Cluster Assignments:", clusters)

--------

Adicionei a função list_files que você forneceu ao código existente e integrei sua utilização no pipeline principal para listar os arquivos antes de carregar os dados. Também inclui a exibição dos arquivos encontrados para facilitar a visualização. Se precisar de mais ajustes ou aprimoramentos, é só avisar!

In [ ]:
import os
import polars as pl
import numpy as np
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from tqdm import tqdm
from glob import glob

# Step 1: List Files
def list_files(dataset_path):
    csv_files_list = []

    folders_list = os.listdir(dataset_path)
    for folder in folders_list:
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            sub_folders_list = os.listdir(folder_path)
            for sub_folder in sub_folders_list:
                sub_folder_path = os.path.join(folder_path, sub_folder)
                if os.path.isdir(sub_folder_path):
                    list_csv = glob(f"{sub_folder_path}/*.csv")
                    for csv_file in list_csv:
                        csv_files_list.append((csv_file, csv_file))
                elif sub_folder_path.endswith(".csv"):
                    csv_files_list.append((sub_folder_path, sub_folder_path))

    return csv_files_list

# Step 2: Load Data
def load_vibration_data(base_path):
    dataset_columns_name = ["tachometer", "underhang_1", "underhang_2", "underhang_3", "overhang_1", "overhang_2", "overhang_3", "microphone"]
    data = []
    labels = []

    for root, dirs, files in os.walk(base_path):
        for file in tqdm(files, desc="Loading files"):
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                df = pl.read_csv(file_path, has_header=True, new_columns=dataset_columns_name)
                data.append(df)
                label = os.path.basename(os.path.dirname(file_path))
                labels.append(label)

    return data, labels

# Step 3: Extract Features
def extract_features(df, sampling_rate):
    signal = df["microphone"].to_numpy()

    # Basic Statistics
    mean = np.mean(signal)
    std_dev = np.std(signal)
    min_val = np.min(signal)
    max_val = np.max(signal)
    kurtosis = pl.Series(signal).kurtosis()
    skewness = pl.Series(signal).skew()

    # Frequency Domain Features
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1 / sampling_rate)
    fft_values = np.abs(fft(signal))
    dominant_freq = freq[np.argmax(fft_values)]
    energy = np.sum(fft_values ** 2)

    # Time Domain Features
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zero_crossing_rate = len(zero_crossings) / len(signal)

    return {
        "mean": mean,
        "std_dev": std_dev,
        "min": min_val,
        "max": max_val,
        "kurtosis": kurtosis,
        "skewness": skewness,
        "dominant_frequency": dominant_freq,
        "frequency_energy": energy,
        "zero_crossing_rate": zero_crossing_rate,
    }

# Step 4: Build Feature Matrix
def build_feature_matrix(data, sampling_rate):
    features = []
    for df in tqdm(data, desc="Extracting features"):
        features.append(extract_features(df, sampling_rate))

    return pl.DataFrame(features)

# Step 5: Preprocessing and Clustering
def cluster_data(features):
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Dimensionality Reduction (optional)
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(scaled_features)

    # DBSCAN Clustering
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    clusters = dbscan.fit_predict(reduced_features)

    return reduced_features, clusters

# Step 6: Visualize Clusters
def visualize_clusters(reduced_features, clusters):
    import matplotlib.pyplot as plt

    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap="viridis")
    plt.colorbar(label="Cluster Label")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("DBSCAN Clustering Results")
    plt.show()

# Main Pipeline
if __name__ == "__main__":
    # Base path to data
    base_path = "Data"  # Update with the correct path

    # Sampling rate (update as needed)
    sampling_rate = 50000

    # List Files
    file_list = list_files(base_path)
    print("Files found:", file_list)

    # Load Data
    data, labels = load_vibration_data(base_path)

    # Build Feature Matrix
    features = build_feature_matrix(data, sampling_rate)

    # Clustering
    reduced_features, clusters = cluster_data(features)

    # Visualize Results
    visualize_clusters(reduced_features, clusters)

    # Display cluster assignment
    print("Cluster Assignments:", clusters)


Poderia gerar um segundo plot no final, onde são apontados os rótulos, nas listas retornadas pelo list_files há tuplas elas indicam:
(caminho_do_arquivo, nome_do_arquivo). Todo arquivo que tenha a palavra "normal"  é uma medição normal, caso contrário, é uma medição onde há anomalia

--------

Adicionei um segundo gráfico que classifica os arquivos como "normal" ou "anomaly" com base no nome do arquivo e os exibe usando diferentes cores. Certifique-se de que o conjunto de dados contém os nomes esperados para que os rótulos sejam atribuídos corretamente. Se precisar de mais ajustes, é só avisar!

In [ ]:
import os
import polars as pl
import numpy as np
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from tqdm import tqdm
from glob import glob

# Step 1: List Files
def list_files(dataset_path):
    csv_files_list = []

    folders_list = os.listdir(dataset_path)
    for folder in folders_list:
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            sub_folders_list = os.listdir(folder_path)
            for sub_folder in sub_folders_list:
                sub_folder_path = os.path.join(folder_path, sub_folder)
                if os.path.isdir(sub_folder_path):
                    list_csv = glob(f"{sub_folder_path}/*.csv")
                    for csv_file in list_csv:
                        csv_files_list.append((csv_file, csv_file))
                elif sub_folder_path.endswith(".csv"):
                    csv_files_list.append((sub_folder_path, sub_folder_path))

    return csv_files_list

# Step 2: Load Data
def load_vibration_data_from_list(file_list):
    dataset_columns_name = ["tachometer", "underhang_1", "underhang_2", "underhang_3", "overhang_1", "overhang_2", "overhang_3", "microphone"]
    data = []
    labels = []

    for file_path, label in tqdm(file_list, desc="Loading files"):
        df = pl.read_csv(file_path, has_header=True, new_columns=dataset_columns_name)
        data.append(df)
        labels.append(label)

    return data, labels

# Step 3: Extract Features
def extract_features(df, sampling_rate):
    signal = df["microphone"].to_numpy()

    # Basic Statistics
    mean = np.mean(signal)
    std_dev = np.std(signal)
    min_val = np.min(signal)
    max_val = np.max(signal)
    kurtosis = pl.Series(signal).kurtosis()
    skewness = pl.Series(signal).skew()

    # Frequency Domain Features
    N = len(signal)
    freq = np.fft.fftfreq(N, d=1 / sampling_rate)
    fft_values = np.abs(fft(signal))
    dominant_freq = freq[np.argmax(fft_values)]
    energy = np.sum(fft_values ** 2)

    # Time Domain Features
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    zero_crossing_rate = len(zero_crossings) / len(signal)

    return {
        "mean": mean,
        "std_dev": std_dev,
        "min": min_val,
        "max": max_val,
        "kurtosis": kurtosis,
        "skewness": skewness,
        "dominant_frequency": dominant_freq,
        "frequency_energy": energy,
        "zero_crossing_rate": zero_crossing_rate,
    }

# Step 4: Build Feature Matrix
def build_feature_matrix(data, sampling_rate):
    features = []
    for df in tqdm(data, desc="Extracting features"):
        features.append(extract_features(df, sampling_rate))

    return pl.DataFrame(features)

# Step 5: Preprocessing and Clustering
def cluster_data(features):
    # Standardize features
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Dimensionality Reduction (optional)
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(scaled_features)

    # DBSCAN Clustering
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    clusters = dbscan.fit_predict(reduced_features)

    return reduced_features, clusters

# Step 6: Visualize Clusters
def visualize_clusters(reduced_features, clusters):
    import matplotlib.pyplot as plt

    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=clusters, cmap="viridis")
    plt.colorbar(label="Cluster Label")
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("DBSCAN Clustering Results")
    plt.show()

def visualize_labels(file_list, clusters):
    import matplotlib.pyplot as plt

    labels = ["normal" if "normal" in name.lower() else "anomaly" for _, name in file_list]
    color_map = {"normal": "blue", "anomaly": "red"}
    colors = [color_map[label] for label in labels]

    plt.scatter(clusters[:, 0], clusters[:, 1], c=colors, label=labels, alpha=0.6)
    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("Visualization by Labels (Normal vs Anomaly)")
    plt.show()

# Main Pipeline
if __name__ == "__main__":
    # Base path to data
    base_path = "Data"  # Update with the correct path

    # Sampling rate (update as needed)
    sampling_rate = 50000

    # List Files
    file_list = list_files(base_path)
    print("Files found:", file_list)

    # Load Data
    data, labels = load_vibration_data_from_list(file_list)

    # Build Feature Matrix
    features = build_feature_matrix(data, sampling_rate)

    # Clustering
    reduced_features, clusters = cluster_data(features)

    # Visualize Results
    visualize_clusters(reduced_features, clusters)

    # Second Plot: Visualization by Labels
    visualize_labels(file_list, reduced_features)

    # Display cluster assignment
    print("Cluster Assignments:", clusters)
